# NEUROTOOL: SPINDLE AND SLOW WAVE MODULATION ANALYSIS
This notebook is a pipeline to use Neurotool for analyzing the modulation between slow waves and spike trains in single channel data. 

__NOTE: Be careful with the memory while using this tool, it might be a good idea to allocate some parts of SSD to be used like RAM so that you don't get a MemoryError.__

## 1. Import the packages required for running the script

Please run the block of code to import the Python packages that are required for running the rest of this script. Make sure that the following files are in the same directory with this notebook:

- analysis_utils
- slowwave_analysis_utils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#import ipywidgets
#from ipywidgets import Layout, HBox, VBox
#from IPython.display import display
#import h5py
import scipy
import scipy.io as spio
from analysis_utils import *
from slowwave_analysis_utils import *
#from extract_data import *

## 2. Entering parameters for data

Please enter the values for the data and spike train that you want to analyze. __This is not a complete tool yet!!!__

In [ ]:
##Name of the data file 

name_html = ipywidgets.HTML(value = "<p><b>Name of the single channel data:</b><br />Enter the name of the raw data file that is in data folder in this directory.</p>")
name = ipywidgets.Text(value = "", placeholder = "Enter name of the file", disabled = False)
display(VBox([name_html, name]))

##File format
ff_html = ipywidgets.HTML(value = "<p><b>File format of data:</b><br />(dat for .dat, cont for .continuous, rhd for .rhd)</p>")
ff = ipywidgets.Text(value = 'dat', placeholder = 'Enter file format for data',
             disabled = False)
display(VBox([ff_html,ff]))

#Units of the data

unit_html = ipywidgets.HTML(value = "<b>Units of the data:</b>")
unit = ipywidgets.Dropdown(options=['Volts', 'MiliVolts', 'MicroVolts'], 
                   value = 'Volts',  disabled = False)
display(VBox([unit_html, unit]))

##file name of the spike train

st_html = ipywidgets.HTML(value = "<p><b>Name of the spike train data file:</b><br />Enter the name of the spike train data file that is in data folder in this directory.</p>")
st = ipywidgets.Text(value = "", placeholder = "Enter name of the file", disabled = False)
display(VBox([st_html, st]))

##File format
ffst_html = ipywidgets.HTML(value = "<p><b>File format of spike train:</b><br />(dat for .dat, cont for .continuous, rhd for .rhd)</p>")
ffst = ipywidgets.Text(value = 'dat', placeholder = 'Enter file format for data',
             disabled = False)
display(VBox([ffst_html,ffst]))

##Sampling frequency of the data

fs_html = ipywidgets.HTML(value = "<p><b>Sampling frequency of the data:</b><br />Enter the sampling frequency of the recording.</p>")
fs = ipywidgets.Text(value = "", placeholder = "Enter sampling frequency", disabled = False)
display(VBox([fs_html, fs]))

## 3. Obtaining the dataset

Please run the following script obtain the parameters in the dataset.

In [ ]:
# script to obtain single channel data, its spike train, and sampling frequency
#data = get_data(name.value, ff.value, unit.value)
#spike_train = get_spike_train_data(st.value, ffst.value)
noc = 72
fs = 1250
sf = 30000
raw_data = np.fromfile('BWRat17_121712.eeg', 'int16')
num_samples = int(len(raw_data)/ noc)
data = np.transpose(np.reshape(raw_data, [num_samples, noc]))[12][0:5000000]
l = len(data)
times0 = np.arange(l)
times0 = times0 / fs
del raw_data
data0 = data
data = signal.resample(data,int(sf*l*(1/fs)))
l = len(data)
l0 = len(data0)
times = np.arange(l)
times = times / sf

In [ ]:
stable_spike_matlab = spio.loadmat("BWRat17_121712_SStable.mat")
num_of_units = len(stable_spike_matlab["S_CellFormat"][0])
peak_index = []
spike_train = []

for unit_index in range(num_of_units):
    data_add = []
    for j in stable_spike_matlab["S_CellFormat"][0][unit_index]:
        if j * sf < l:
            data_add.append(int(j * sf))
    peak_index.append(data_add)
    spike_train.append(get_spike_train(peak_index[unit_index], l))
del data_add
del peak_index
del stable_spike_matlab

In [ ]:
spike_train = spike_train[46:48] # taking 2 units for demonstration

In the code above, we obtained the LFP data and spike trains from our files. When you want to work with your own data you should extract your data from your files as a numpy array. Here because we had spike trains found in a different sampling frequency, we upsampled the data for further use, however this will not be neccessary when you want to work on your own data. Every element in the spike train data is the spike train of each unit as an array of same length with data. We only took the unit number 46 for demonstration purposes.

## 4. Plotting the data and spike train in a specific time range

Default time range is all data, but it is recommended to be changed.

In [ ]:
%matplotlib notebook
plot_data(data0, times0, 'Time (seconds)', 'Amplitude (uV)','Raw Data', [times0.min(), times0.max()])

In [ ]:
%matplotlib notebook
plot_spike_train(spike_train[1], times, [times.min(), times.max()])

## 5. Finding slow waves inside the data

Here you can change the threshold values for the detection algorithm if you want to. More on algorithm can be found [here](https://github.com/raphaelvallat/yasa).

In [ ]:
#THIS MIGHT TAKE TIME DEPENDING ON SIZE OF THE DATA AND YOUR COMPUTER
sw = find_slowwave(data0, fs)
sw.round(2)

## 6. Plotting slow waves on top of the data in a specific time range

Default time range is all data, but it is recommended to be changed.

In [ ]:
%matplotlib notebook
plot_slowwave(data0, times0, fs, sw, 'Time (seconds)', 'Amplitude (uV)', 'Data with slow waves', [times.min(), times.max()])

In [ ]:
del data0
del times0

## 7. Saving each slow wave raw data

Here we will save the raw data for each slow wave, in the following format: {'0': [slowwave0rawdata], '1': [slowwave1rawdata], ....}

In [ ]:
slowwave_dict = save_slowwave(data0, fs, sw)

## 8. Calculating and saving firing histogram of each unit before, during and after slow wave
This function will output the following:

For each unit:

[Average firing rate of the unit in the following time interval [start - dt:start], Average firing rate of the unit in the following time interval [slowwave duration], Average firing rate of the unit in the following time interval [end : end + dt]]

You can access this information by indexing the output array[unit_index]

In [ ]:
#THIS MIGHT TAKE TIME DEPENDING ON SIZE OF THE DATA AND YOUR COMPUTER
dt = 2
unit_sw_firing = firing_rate_sw(sw, spike_train, sf, dt)

## 9. Calculating the slow wave phase histograms and time/phase aligned slow wave spike trains and saving all the data
This function takes the spike train of all units, its samplng frequency, slow wave dataFrame, and phase intervals as the input
in order to output the following:
[num_spikes, phase_hist_spike_trains, time_spike_trains, unit_hist_arrays]

1. num_spikes is an array showing how many spikes in total are found in slow waves, for each unit. You can access the data by indexing the array with unit_index
2. phase_hist_spike_trains is a dictionary which holds the phase histograms of each slow wave, for each unit. To access the data, first you should index the
unit_index in dictionary, (e.g. arr[str(unit_index)]) then the slow wave number.
3. time_spike_trains is a dictionary which holds time aligned spike trains in slow waves, for each unit. Data can be accessed as demonstrated in 
phase_hist_spike_trains explanation
4. unit_hist_arrays is a dictionary holding sums of the phase histogram arrays for all slow waves, for each unit. 
You can access the data by indexing the array with unit_index

Here you can change the phase intervals of the histogram by changing the variable, 'phase'. 

In [ ]:
#THIS MIGHT TAKE TIME DEPENDING ON SIZE OF THE DATA AND YOUR COMPUTER
phase = np.pi/6
sw_allunits = sw_all_units(sw, phase, sf, spike_train)

num_spikes = sw_allunits[0]
phase_hist_spike_trains = sw_allunits[1]
time_spike_trains = sw_allunits[2]
unit_hist_arrays = sw_allunits[3]

del sw_allunits

## 10. Calculating slow wave phase histogram of a spike train of a specific unit

Using the following functions, you can directly calculate the phase_hist_spike_train, time_spike_train, and unit_hist_array of a specific unit by indexing its unit index for spike train. See the notebook files to understand outputs of these functions.

- slowwave_phase_hist(sp, phase, fs, spike_train[unit_index])

## 11. Saved results

At the end of this analysis, we have the following dictionaries:

- slowwave_dict
- num_spikes
- unit_sw_firing
- phase_hist_spike_trains
- time_spike_trains
- unit_hist_arrays

Each storing the following information, respectively:

- Each slow wave's raw data in slow wave time window
- Number of spikes that are found in slow waves for each unit
- Firing rate for each unit before, during, and after slow waves
- Phase aligned spike trains of each unit in each slow wave
- Time aligned spike trains of each unit in each slow wave
- Sum of the all phase aligned spike trains in all slow waves for each unit


You can access these variables for any purpose. Run the following cells to print the data inside them.

In [ ]:
slowwave_dict

In [ ]:
num_spikes

In [ ]:
unit_sw_firing

In [ ]:
phase_hist_spike_trains

In [ ]:
time_spike_trains

In [ ]:
unit_hist_arrays

## 12. Plotting slow wave's phase histogram

Here's an example of a slow wave phase histogram, with unit index 46.

In [ ]:
%matplotlib notebook
plot_barchart(unit_hist_arrays['0'], phase)

In [ ]:
%matplotlib notebook
plot_polarbarchart(unit_hist_arrays['0'], phase)

## 13. Conclusion
After all, looking at these histogram plots we can understand if there's a modulation between spiking pattern and slow waves. 

This notebook is created by Mert Unsal in ETH Zurich Neuroinformatics Lab under supervision of PhD. Tansel Baran Yasar. You can reach me out at mailmertunsal@gmail.com for further questions or discussion.